# Poultry Disease Classification from Fecal Images

This notebook classifies poultry diseases based on fecal images into four classes:

- **Healthy**
- **Coccidiosis (cocci)**
- **Salmonella (salmo)**
- **Newcastle Disease (ncd)**

> ⚙️ *The model is optimized for Mac M1 with 8GB RAM.* (Farhan Mashrur)
- Model initially developed with Ahmed Abdulla (Teammate) for Mac M2 


### GPU availability Testing :

In [45]:
import tensorflow as tf
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


TensorFlow version: 2.16.2
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available: 1


 ## 1. Importing Libraries


In [46]:
import os
import time
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Enable mixed precision for Apple Silicon (M1/M2)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print('Modules loaded')

Modules loaded


## 2. Enable GPU Acceleration (for Mac M1)
#### For Mac M1, we are using TensorFlow-MacOS and Metal plugin are installed.

In [47]:
try:
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU acceleration enabled on M1")
    else:
        print("No GPU found")
except Exception as e:
    print("GPU acceleration not available:", e)


# 3. Image Settings for Mac M1 (8GB RAM)
IMG_SIZE = (160, 160)  # Reduced for efficiency
CHANNELS = 3
BATCH_SIZE = 32        # Lower to 16 if memory issues occur
IMG_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], CHANNELS)

GPU acceleration enabled on M1


## 3) Custom Callback for training and Monitoring


In [52]:
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras

class MyCallback(keras.callbacks.Callback):
    def __init__(self, model, patience=1, stop_patience=3, threshold=0.9, factor=0.5, batches=None, epochs=None):
        super(MyCallback, self).__init__()
        self._model = model
        self.patience = patience 
        self.stop_patience = stop_patience
        self.threshold = threshold
        self.factor = factor
        self.batches = batches
        self.epochs = epochs
        
        self.count = 0
        self.stop_count = 0
        self.best_epoch = 1
        
        try:
            self.current_lr = 0.001
            if hasattr(model.optimizer, 'learning_rate'):
                lr = model.optimizer.learning_rate
                if hasattr(lr, 'numpy'):
                    self.current_lr = float(lr.numpy())
            elif hasattr(model.optimizer, 'lr'):
                lr = model.optimizer.lr
                if hasattr(lr, 'numpy'):
                    self.current_lr = float(lr.numpy())
        except:
            self.current_lr = 0.001
            
        self.initial_lr = self.current_lr
        self.highest_tracc = 0.0
        self.lowest_vloss = np.inf
        self.best_weights = self._model.get_weights()
        self.initial_weights = self._model.get_weights()

        print("✅ Callback initialization complete.")

    def on_train_begin(self, logs=None):
        msg = '{0:^8s}{1:^10s}{2:^9s}{3:^9s}{4:^9s}{5:^9s}{6:^9s}{7:^10s}{8:10s}{9:^8s}'.format(
            'Epoch', 'Loss', 'Accuracy', 'V_loss', 'V_acc', 'LR', 'Next LR', 'Monitor','% Improv', 'Duration')
        print(msg)
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        stop_time = time.time()
        tr_duration = stop_time - self.start_time
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'\nTraining time: {int(hours)}h {int(minutes)}m {seconds:.2f}s'
        print(msg)
        self._model.set_weights(self.best_weights)
        print("✅ Best weights restored.")

    def on_train_batch_end(self, batch, logs=None):
        acc = logs.get('accuracy') * 100
        loss = logs.get('loss')
        msg = f'processing batch {batch + 1} of {self.batches} - accuracy: {acc:.2f}% - loss: {loss:.5f}'
        print(msg, '\r', end='')

    def on_epoch_begin(self, epoch, logs=None):
        self.ep_start = time.time()
        print(f"\n🔁 Starting epoch {epoch + 1}")

    def _update_lr(self, new_lr):
        try:
            if hasattr(self._model.optimizer, 'learning_rate'):
                tf.keras.backend.set_value(self._model.optimizer.learning_rate, new_lr)
            elif hasattr(self._model.optimizer, 'lr'):
                tf.keras.backend.set_value(self._model.optimizer.lr, new_lr)
            self.current_lr = new_lr
            print(f"📉 Learning rate updated to {new_lr:.6f}")
        except Exception as e:
            print("⚠️ Failed to update learning rate:", e)
        return new_lr

    def on_epoch_end(self, epoch, logs=None):
        ep_end = time.time()
        duration = ep_end - self.ep_start
        current_lr = self.current_lr

        acc = logs.get('accuracy')
        v_acc = logs.get('val_accuracy')
        loss = logs.get('loss')
        v_loss = logs.get('val_loss')
        next_lr = current_lr

        if acc < self.threshold:
            monitor = 'accuracy'
            pimprov = 0.0 if epoch == 0 else (acc - self.highest_tracc) * 100 / self.highest_tracc
            if acc > self.highest_tracc:
                self.highest_tracc = acc
                self.best_weights = self._model.get_weights()
                self.count = 0
                self.stop_count = 0
                if v_loss < self.lowest_vloss:
                    self.lowest_vloss = v_loss
                self.best_epoch = epoch + 1
            else:
                if self.count >= self.patience - 1:
                    next_lr = current_lr * self.factor
                    self._update_lr(next_lr)
                    self.count = 0
                    self.stop_count += 1
                    if v_loss < self.lowest_vloss:
                        self.lowest_vloss = v_loss
                else:
                    self.count += 1
        else:
            monitor = 'val_loss'
            pimprov = 0.0 if epoch == 0 else (self.lowest_vloss - v_loss) * 100 / self.lowest_vloss
            if v_loss < self.lowest_vloss:
                self.lowest_vloss = v_loss
                self.best_weights = self._model.get_weights()
                self.count = 0
                self.stop_count = 0
                self.best_epoch = epoch + 1
            else:
                if self.count >= self.patience - 1:
                    next_lr = current_lr * self.factor
                    self._update_lr(next_lr)
                    self.stop_count += 1
                    self.count = 0
                else:
                    self.count += 1
                if acc > self.highest_tracc:
                    self.highest_tracc = acc

        msg = f'{epoch + 1:^8} {loss:^10.3f}{acc * 100:^9.2f}{v_loss:^9.5f}{v_acc * 100:^9.2f}{current_lr:^9.5f}{next_lr:^9.5f}{monitor:^11s}{pimprov:^10.2f}{duration:^8.2f}'
        print(msg)

        if self.stop_count > self.stop_patience - 1:
            print(f"\n🛑 Training halted at epoch {epoch + 1} — no improvement after {self.stop_patience} learning rate adjustments.")
            self.model.stop_training = True




## Code to test callback functions


In [53]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Assume your callback class is already defined above as MyCallback

# 1. Create dummy image data (100 samples of 32x32 RGB images)
X_dummy = np.random.rand(100, 32, 32, 3).astype(np.float32)

# 2. Create dummy labels (4 classes)
y_dummy = np.random.randint(0, 4, 100)

# 3. Build a simple model
model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# 4. Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. Create your custom callback instance
cb = MyCallback(model=model, epochs=5, batches=X_dummy.shape[0] // 16)

# 6. Train with the dummy data
model.fit(X_dummy, y_dummy,
          epochs=5,
          batch_size=16,
          validation_split=0.2,
          callbacks=[cb])


✅ Callback initialization complete.
 Epoch     Loss   Accuracy  V_loss    V_acc     LR     Next LR  Monitor  % Improv  Duration

🔁 Starting epoch 1
Epoch 1/5
   1       1.934     25.00   1.97165   20.00   0.00100  0.00100  accuracy     0.00     1.19   loss: 1.5473processing batch 2 of 6 - accuracy: 28.12% - loss: 1.61060 
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.2526 - loss: 1.7395 - val_accuracy: 0.2000 - val_loss: 1.9716

🔁 Starting epoch 2
Epoch 2/5
⚠️ Failed to update learning rate: 'str' object has no attribute 'name' - accuracy: 0.2044 - loss: 2.2094processing batch 2 of 6 - accuracy: 21.88% - loss: 2.19255 processing batch 5 of 6 - accuracy: 18.75% - loss: 2.03662 
   2       2.037     18.75   1.70025   30.00   0.00100  0.00050  accuracy    -25.00    0.15  
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1988 - loss: 2.1518 - val_accuracy: 0.3000 - val_loss: 1.7002

🔁 Starting epoch 3
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2313 - loss: 1.763